<a href="https://colab.research.google.com/github/nephelim74/machinelearn/blob/main/ml_dz9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Импорт необходимых библиотек

In [20]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.datasets import load_wine
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier as SklearnRandomForestClassifier

2. Загрузка набора данных
* Для примера используем набор данных Iris.

In [21]:
data = load_digits()
X = data.data  # Признаки
y = data.target  # Целевая переменная

# Преобразуем в DataFrame для удобства
df = pd.DataFrame(X, columns=data.feature_names)
df['target'] = y

3. Предварительная обработка данных
* Масштабируем признаки и разделим данные на обучающую и тестовую выборки.

In [22]:
# Масштабирование признаков
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

4. Реализация алгоритма случайного леса
* Реализуем базовый случайный лес с использованием деревьев решений.

In [23]:
from sklearn.tree import DecisionTreeClassifier

class RandomForestClassifier:
  #  Инициализация класса (__init__)
    def __init__(self, n_estimators=100, max_features='sqrt', random_state=None):
        self.n_estimators = n_estimators
        self.max_features = max_features
        self.random_state = random_state
        self.trees = []
        '''
        n_estimators=100: Количество деревьев в лесу. По умолчанию создается 100 деревьев.
        max_features='sqrt': Количество признаков, которые случайно выбираются для каждого дерева.'
        По умолчанию используется квадратный корень из общего числа признаков (sqrt).
        random_state=None: Зерно для генератора случайных чисел, чтобы обеспечить воспроизводимость результатов.
        self.trees = []: Список для хранения обученных деревьев и их параметров.
        '''
    # Метод fit (обучение модели)
    def fit(self, X, y):
        np.random.seed(self.random_state)
        n_samples, n_features = X.shape
        self.feature_indices = []  # Для хранения индексов выбранных признаков
        '''
        np.random.seed(self.random_state): Устанавливает seed для генератора случайных чисел, чтобы результаты были воспроизводимыми.
        n_samples, n_features = X.shape: Определяет количество строк (образцов) и столбцов (признаков) в данных.
        self.feature_indices = []: Список для хранения индексов признаков, выбранных для каждого дерева.
        '''
        # Цикл для создания и обучения деревьев:
        for _ in range(self.n_estimators):
            # Выбор случайных признаков
            if self.max_features == 'sqrt':
                n_selected_features = int(np.sqrt(n_features))
            else:
                n_selected_features = self.max_features
            feature_indices = np.random.choice(n_features, n_selected_features, replace=False)
            self.feature_indices.append(feature_indices)
            '''
            for _ in range(self.n_estimators):: Цикл, который создает и обучает n_estimators деревьев.
            if self.max_features == 'sqrt':: Если max_features равно 'sqrt', то количество случайно выбираемых признаков равно квадратному корню из общего числа признаков.
            n_selected_features = int(np.sqrt(n_features)): Вычисляет количество признаков для выбора.
            feature_indices = np.random.choice(n_features, n_selected_features, replace=False): Случайно выбирает индексы признаков без повторений.
            self.feature_indices.append(feature_indices): Сохраняет выбранные индексы признаков для текущего дерева.
            '''
            # Выбор случайных samples с повторением (bootstrap)
            sample_indices = np.random.choice(n_samples, n_samples, replace=True)
            X_subset = X[sample_indices][:, feature_indices]
            y_subset = y[sample_indices]
            '''
            sample_indices = np.random.choice(n_samples, n_samples, replace=True): Случайно выбирает индексы строк (образцов) с повторением (bootstrap sampling).
            X_subset = X[sample_indices][:, feature_indices]: Создает подмножество данных для обучения текущего дерева, используя выбранные строки и признаки.
            y_subset = y[sample_indices]: Создает подмножество целевых значений для выбранных строк.
            '''
            # Обучение дерева
            tree = DecisionTreeClassifier(random_state=self.random_state)
            tree.fit(X_subset, y_subset)
            self.trees.append((tree, feature_indices))
            '''
            tree = DecisionTreeClassifier(random_state=self.random_state): Создает объект дерева решений.
            tree.fit(X_subset, y_subset): Обучает дерево на подмножестве данных.
            self.trees.append((tree, feature_indices)): Сохраняет обученное дерево и индексы признаков, использованных для его обучения.
            '''
    #  Метод predict (предсказание)
    def predict(self, X):
        predictions = np.zeros((X.shape[0], len(self.trees)))
        # Создает массив для хранения предсказаний каждого дерева. Размер массива: (количество образцов) x (количество деревьев).
        # Предсказание для каждого дерева:
        for i, (tree, feature_indices) in enumerate(self.trees):
            X_subset = X[:, feature_indices]
            predictions[:, i] = tree.predict(X_subset)
            '''
            for i, (tree, feature_indices) in enumerate(self.trees):: Проходит по всем обученным деревьям.
            X_subset = X[:, feature_indices]: Выбирает признаки, которые использовались для обучения текущего дерева.
            predictions[:, i] = tree.predict(X_subset): Сохраняет предсказания текущего дерева в массив.
            '''

        # Голосование большинством
        return np.array([np.bincount(predictions[i].astype(int)).argmax() for i in range(X.shape[0])])
        '''
        np.bincount(predictions[i].astype(int)): Подсчитывает количество голосов для каждого класса для каждого образца.
        .argmax(): Выбирает класс с наибольшим количеством голосов.
        return np.array([...]): Возвращает итоговые предсказания для всех образцов.
        '''

5. Обучение модели
* Обучим нашу реализацию случайного леса.


In [24]:
# Создание и обучение модели
rf_custom = RandomForestClassifier(n_estimators=100, max_features='sqrt', random_state=42)
rf_custom.fit(X_train, y_train)

6. Оценка производительности модели
* Оценим качество модели на тестовой выборке.

In [25]:
# Предсказание на тестовой выборке
y_pred_custom = rf_custom.predict(X_test)

# Метрики качества
accuracy = accuracy_score(y_test, y_pred_custom)
precision = precision_score(y_test, y_pred_custom, average='weighted')
recall = recall_score(y_test, y_pred_custom, average='weighted')
f1 = f1_score(y_test, y_pred_custom, average='weighted')

print(f"Custom Random Forest Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Custom Random Forest Metrics:
Accuracy: 0.9537
Precision: 0.9537
Recall: 0.9537
F1 Score: 0.9534



7. Сравнение с реализацией из scikit-learn
* Теперь обучим стандартный RandomForestClassifier из scikit-learn и сравним результаты.

In [26]:
# Обучение модели из scikit-learn
rf_sklearn = SklearnRandomForestClassifier(n_estimators=100, max_features='sqrt', random_state=42)
rf_sklearn.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred_sklearn = rf_sklearn.predict(X_test)

# Метрики качества
accuracy_sklearn = accuracy_score(y_test, y_pred_sklearn)
precision_sklearn = precision_score(y_test, y_pred_sklearn, average='weighted')
recall_sklearn = recall_score(y_test, y_pred_sklearn, average='weighted')
f1_sklearn = f1_score(y_test, y_pred_sklearn, average='weighted')

print(f"\nScikit-learn Random Forest Metrics:")
print(f"Accuracy: {accuracy_sklearn:.4f}")
print(f"Precision: {precision_sklearn:.4f}")
print(f"Recall: {recall_sklearn:.4f}")
print(f"F1 Score: {f1_sklearn:.4f}")


Scikit-learn Random Forest Metrics:
Accuracy: 0.9741
Precision: 0.9741
Recall: 0.9741
F1 Score: 0.9740


### Выводы:
Датасеты Iris, слишком простые и дают резульаты равные 1.
Поэтому используем более сложный digits.
Результаты практиески близкие к Scikit-learn

